In [ ]:
from bs4 import BeautifulSoup
import requests,time
import pandas as pd



Run below code to scrape data from website

In [ ]:
base_url = "https://www.stproperty.sg"
url = "https://www.stproperty.sg/singapore-property-listings/hdb-for-sale"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")

In [ ]:
last_page = int(doc.find_all(class_ = "hidden-xs")[-1].text.strip())

In [ ]:
def find_all_hdb_in_page(doc):
    tags = doc.find_all(class_ = "listingContainer")
    detailed_urls = [];
    for tag in tags:
        url = tag.find(class_ = "listingDetailsDivLink")
        url = url.attrs['href']
        detailed_urls.append(base_url + url)
    return detailed_urls


In [ ]:
def format_price(price):
    price = price.text.strip()
    price = price.replace(",", "")
    return int(price)

In [ ]:
def get_description(url): 
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    try:
        town = doc.find(text = "HDB Town")
        town = town.parent.find_next_sibling().text
    except Exception:
        town = None;
    price = doc.find(class_ = "listing-price")
    price =  format_price(price.find('span', {"itemprop":"price"}))
    size = doc.find(text = "Size")
    size = size.parent.find_next_sibling().find("span", {"itemprop":"value"}).text
    size = int(size.split()[0].replace(",",""))
    try: 
        bedrooms = doc.find(text= 'Bedrooms')
        bedrooms = int(bedrooms.parent.find_next_sibling().text)
    except ValueError:
        bedrooms = 0
        print(town)
    except AttributeError:
        bedrooms = 0
    # if no bathrooms on website, assign bathrooms to be one
    try:
        bathrooms = doc.find(text = "Bathrooms")
        bathrooms = int(bathrooms.parent.find_next_sibling().text)
    except AttributeError:
        bathrooms = 0
    except ValueError:
        bathrooms = 0
    details = {"town" : town, "price": price, "size": size , "bedrooms":bedrooms, 'bathrooms':bathrooms}
    return details

In [ ]:
data = []

In [ ]:
i = 1;
for i in range(0, 121):
    next_page_url = f"/?page={i}"
    updated_url = url + next_page_url
    print(updated_url)
    result = requests.get(updated_url);
    doc = BeautifulSoup(result.text, "html.parser")
    detailed_url = find_all_hdb_in_page(doc)
    print(len(detailed_url))
    for u in detailed_url:
        time.sleep(1)
        data.append(get_description(u))
    



In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("hdb_details.csv", index=False)